In [1]:
import io
import json
import pickle
import sys
import time
from collections import defaultdict
from datetime import datetime

import dotenv
# import ipydatagrid
# from ipydatagrid import datagrid
import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import openai
import pandas as pd

# import qgrid
import scipy
import seaborn as sns
import statsmodels.api as sm
import ydata_profiling
from IPython.display import Image, Markdown, clear_output, display
from openai import OpenAI
from scipy import stats
from scipy.stats import kurtosis, norm, shapiro, skew
from ydata_profiling import ProfileReport

print(Markdown(f"python          {'.'.join(map(str, sys.version_info[:3]))}")
print(Markdown(f"numpy           {np.__version__}")
print(Markdown(f"pandas          {pd.__version__}")
print(Markdown(f"seaborn         {sns.__version__}")
print(Markdown(f"matplotlib      {mpl.__version__}")
print(Markdown(f"openai          {openai.__version__}")
print(Markdown(f"ydata_profiling {ydata_profiling.__version__}")
print(Markdown(f"ipywidgets      {widgets.__version__}")
# print(Markdown(f"qgrid           {qgrid.__version__}")
# print(Markdown(f"ipydatagrid     {ipydatagrid.__version__}")

%matplotlib inline

python          3.9.19
numpy           1.26.4
pandas          2.2.2
seaborn         0.12.2
matplotlib      3.8.4
openai          1.23.2
ydata_profiling v4.7.0
ipywidgets      8.0.4


In [2]:
dotenv.load_dotenv()

True

In [3]:
client = OpenAI()

In [49]:
prompt = """
I have uploaded a text file called ADHD.txt .

You are a college level instructor with deep knowledge of psychology and the ability to explain complex topics in simple terms
and in a clear, engaging writing style.

Your high-level task is to help create a study guide that will prepare me for reading the book in depth, knowing what to expect, what questions I should think about,
and what to keep in mind as I study the text, and summarizing chapters. 

Assume I am a college graduate or undergraduate who knows little about the topic. Write in a clear, engaging style, simplify technical 
expositions and descriptions, and define any terms that are not common knowledge.

You will only use information contained in the book to answer questions.

To start, what are all the chapters in the book?
"""

In [6]:
file1 = client.files.create(file=open("ADHD.txt", "rb"), purpose="assistants")
print(Markdown(file1)


FileObject(id='file-oUpWecsVE9mdDpurtQvecEv4', bytes=668974, created_at=1713881524, filename='ADHD.txt', object='file', purpose='assistants', status='processed', status_details=None)


In [7]:
# Create a vector store caled "ADHD text"
vector_store = client.beta.vector_stores.create(name="ADHD text")
 
# Ready the files for upload to OpenAI 
file_paths = ["ADHD.txt"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation. 
print(Markdown(file_batch.status)
print(Markdown(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [8]:
assistant = client.beta.assistants.create(
  name="ADHD assistant",
  instructions="""
  You are a college level instructor with deep knowledge of psychology and the ability to explain complex topics in a clear, engaging style. 
  Answer questions using the texts in the knowledge base."
  """,
  model="gpt-4-turbo",
  tools=[{"type": "file_search"}],
)


In [9]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)


In [50]:
# create a new thread of messages
thread = client.beta.threads.create()
thread

Thread(id='thread_AxbG4A5oBzGjBsltc40YbPEr', created_at=1713884294, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [51]:
message = client.beta.threads.messages.create(
    thread_id=thread.id, 
    role="user", 
    content=prompt, 
)
message

Message(id='msg_eVtSR2TrTzgWBLg4j7x8FY5E', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='\nI have uploaded a text file called ADHD.txt .\n\nYou are a college level instructor with deep knowledge of psychology and the ability to explain complex topics in simple terms\nand in a clear, engaging writing style.\n\nYour high-level task is to help create a study guide that will prepare me for reading the book in depth, knowing what to expect, what questions I should think about,\nand what to keep in mind as I study the text, and summarizing chapters. \n\nAssume I am a college graduate or undergraduate who knows little about the topic. Write in a clear, engaging style, simplify technical \nexpositions and descriptions, and define any terms that are not common knowledge.\n\nYou will only use information contained in the book to answer questions.\n\nTo start, what are all the chapters in the book?\n'), type='text')], created_at=1

In [55]:
# using streaming
# seems to stream messages as opposed to tokens

from typing_extensions import override
from openai import AssistantEventHandler

class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        global display_handle
        global message_content
        message_content += f"\nassistant > \n"
        display_handle.update(Markdown(message_content))

    @override
    def on_tool_call_created(self, tool_call):
        global display_handle
        global message_content
        message_content += f"\nassistant > {tool_call.type}\n"
        display_handle.update(Markdown(message_content))
        
    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        global message_content
        global display_handle
        temp_message_content = message.content[0].text
        annotations = temp_message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            temp_message_content.value = temp_message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        message_content += temp_message_content.value
        message_content += "\n".join(citations)
        message_content += "\n\n"
        
        display_handle.update(Markdown(message_content))
        

In [53]:
# Then, we use the stream SDK helper
# with the EventHandler class to create the Run
# and stream the response.

display_handle = display(Markdown(""), display_id=True)
message_content = ""

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as 'my dear reader and friend'. The user has a premium account.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()
    


assistant > file_search

assistant > My dear reader and friend, I'm pleased to assist you in your quest for understanding the text on Attention Deficit Disorder (ADD) that you plan to study in depth. Here are the chapters contained in the book, which will guide your thorough exploration:

1. **What Is Attention Deficit Disorder?**
2. **"I Sang in My Chains Like the Sea"**
3. **"Sequence Ravelled Out of Sound"**
4. **Living and Loving with ADD**
5. **The Big Struggle**
6. **Parts of the Elephant**
7. **How Do I Know if I Have It?**
8. **What Can You Do About It?**
9. **A Local Habitation and a Name**
10. **The Biology of ADD**

Full credit of this list goes to the Table of Contents of the text file you provided on ADD . As you proceed to read these chapters, these titles may serve as a guideline to help you understand the journey you will embark upon within the realm of ADD study. Each chapter, as denoted by its title, focuses on several facets of understanding and living with ADD, from its diagnosis and biological underpinnings to personal and relational impacts. I encourage you to keep these themes in mind as they will surely enrich your comprehension and appreciation of the subject matter.



In [11]:
# no streaming

def show_latest_message(client, thread):
    """display latest message from a thread as markdown"""

    messages = client.beta.threads.messages.list(thread_id=thread.id)

    for obj in messages.data[0].content:
        if obj.type == "text":
            display(Markdown(obj.text.value))
        elif obj.type == "image_file":
            image_data = client.files.content(obj.image_file.file_id)
            image_data_bytes = image_data.read()
            display(Image(data=image_data_bytes))
        else:
            display(Markdown(obj))
    return messages


def get_file_ids(client, thread):
    """get file ids from latest message annotations"""
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    file_ids = []
    for annotation in messages.data[0].content[0].text.annotations:
        file_ids.append(annotation.file_path.file_id)
    return file_ids


def execute_run(client, assistant, thread, sleeptime=5):
    start_time = datetime.now()

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please address the user as 'my dear reader and friend'. The user has a premium account.",
    )

    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        clear_output()
        messages = show_latest_message(client, thread)

        display_handle.update(Markdown("Status:     ", run.status)
        # display_handle.update(Markdown("start time: ", start_time.strftime("%H:%M:%S"))
        time_now = datetime.now()
        display_handle.update(Markdown("Time :      ", time_now.strftime("%H:%M:%S"))
        difference_in_seconds = (time_now - start_time).total_seconds()
        display_handle.update(Markdown("Elapsed:    ", difference_in_seconds, "seconds")

        if run.status == "completed":
            break
        time.sleep(sleeptime)
    return messages



In [12]:
result = execute_run(client, assistant, thread)


My dear reader and friend, the chapters of the book **"ADHD.txt"** can be outlined as follows:

1. **Introduction to the Anchor Edition by Edward M. Hallowell:**
   - Discusses the general perceptions and understandings of Attention Deficit Disorder (ADD).

2. **"I Sang in My Chains Like the Sea":**
   - Explores the expressive and sometimes hidden aspects of ADD.

3. **"Sequence Ravelled Out of Sound":**
   - Looks into the life and experiences of children with ADD.

4. **"Living and Loving with ADD"**
   - Details the impact of ADD on adult relationships and personal life.

5. **"The Big Struggle":"**
   - Addresses ADD within family dynamics and the challenges faced.

6. **"Parts of the Elephant":**
   - Discusses the different subtypes of ADD.

7. **"How Do I Know if I Have It?":**
   - Steps toward the diagnosis of ADD.

8. **"What Can You Do About It?":**
   - Outlines treatments and interventions for managing ADD.

9. **"A Local Habitation and a Name":**
   - Delves into the biology and neuroscientific understanding of ADD .

Remember, each chapter brings a unique perspective on the various elements and implications of ADD, providing a comprehensive overview of the disorder. It would be beneficial for you to approach each chapter with an intent to understand how ADD can manifest differently across different stages of life and within various personal and interpersonal contexts. This understanding will fortify your grasp of the nuances this condition entails.

Status:      completed
Time :       17:57:04
Elapsed:     27.429763 seconds


In [54]:
display_handle = display(Markdown(""), display_id=True)
message_content = ""

for chapter in range(10):
    
    prompt = f"""
Please summarize chapter {chapter+1}. Provide the title, a brief overview, and bullet points for each of the main sections and points discussed.

Bullet points should include all case studies mentioned, behaviors observed, actions taken and therapeutic outcomes; 
theoretical concepts; biological observations; key takeaways and lessons learned in the chapter.

The chapter summary should be between 500 and 1500 words.
"""

    message = client.beta.threads.messages.create(
        thread_id=thread.id, 
        role="user", 
        content=prompt, 
    )

    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please address the user as 'my dear reader and friend'. The user has a premium account.",
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()
    
# result = execute_run(client, assistant, thread)



assistant > file_search

assistant > ### Chapter 1: What Is Attention Deficit Disorder?

**Title:** What Is Attention Deficit Disorder?

**Overview:**  
Chapter 1 of the book introduces the fundamental aspects of Attention Deficit Disorder (ADD), exploring its various manifestations, theoretical underpinnings, and biological observations. It serves as an essential foundation for understanding ADD's impact on individuals' lives and behaviors.

**Main Sections and Key Points:**

1. **Definition and Symptoms of ADD**:
   - ADD is described as a chronic disturbance evidenced by a myriad of symptoms disproportionately severe compared to the general population.
   - Symptoms include difficulty in sustaining attention, excessive activity, and impulsiveness [0].

2. **Theoretical Concepts**:
   - There are several perspectives on understanding ADD, including decreased activation in the brain's reward centers, deficits in the motivation system, and issues with the inhibition and regulation of sensory inputs.
   - Notable concepts discussed are Paul Wender's theories on ADD's link to the brain’s reward systems and Russell Barkley’s views on defects in the motivation system, leading to difficulties in maintaining attention over time [1].

3. **Biological Observations**:
   - ADD is described as a neurobiological dysfunction leading to difficulties in selective attention and self-regulation.
   - The chapter discusses the neurological basis of ADD, emphasizing variations in brain chemistry and structure that influence behavior and cognition [1].
   - Findings from various studies, including those by Dr. Alan Zametkin, reveal differences in brain glucose metabolism between individuals with ADD and those without, providing a biological evidence of the disorder [3].

4. **Case Studies and Personal Accounts**:
   - The chapter contains narratives illustrating how ADD affects daily life. This includes stories like Jim’s encounters at work, where he lost his job due to symptoms related to ADD.
   - These narratives help contextualize the symptoms and impact of ADD on personal relationships and professional settings [4].

5. **Key Takeaways and Lessons Learned**:
   - ADD is a complex disorder with a variety of manifestations. Understanding it requires careful attention to both its psychological and biological components.
   - Treatment and management strategies are varied, indicating the need for personalized approaches based on individual symptoms and lifestyle requirements.
   - The importance of recognizing ADD early and strategies for management and coping in both personal and professional lives are emphasized [1].

**Conclusion:**  
Chapter 1 of the book provides an essential overview of ADD, blending theoretical, biological, and narrative elements to furnish a multi-faceted understanding of the disorder. This foundation is critical for anyone seeking to understand or manage ADD in themselves or others, making it clear that while challenging, ADD can be effectively managed with the right knowledge and strategies. 

This summary forms the bedrock of our study guide, preparing you for deeper exploration in subsequent chapters, each of which will build on this foundational knowledge to explore more specific elements of living with and managing ADD.[0] ADHD.txt
[1] ADHD.txt
[2] ADHD.txt
[3] ADHD.txt
[4] ADHD.txt
[5] ADHD.txt


assistant > file_search

assistant > ### Chapter 2: "I Sang in My Chains Like the Sea"

**Title:** "I Sang in My Chains Like the Sea"

**Overview:**  
Chapter 2 dwells on the foundational recognition of Attention Deficit Disorder (ADD), emphasizing historical perspectives on how the disorder was perceived and treated. It discusses the shift from seeing ADD symptoms as purely behavioral and moral issues to understanding them as medical conditions with biological underpinnings.

**Main Sections and Key Points:**

1. **Historical Recognition of ADD Symptoms**: 
   - Initially, signs associated with ADD were seen purely as moral and behavioral issues rather than medical conditions.
   - Pioneers like George Frederic Still and William James began challenging old notions by considering neurological explanations for these behaviors [0].

2. **Psychological and Neuropsychological Understanding**:
   - In the early 20th century, the link between ADD symptoms and their possible biological origins began to be explored and acknowledged.
   - The Neuropsychological model was developed thanks to encephalitis outbreaks which helped pin behavioral symptoms to biological effects on the brain [0].

3. **Pharmacological Discoveries**:
   - The counterintuitive use of stimulants like Benzedrine and Ritalin on hyperactive behaviors marked a turning point in the treatment of ADD.
   - These drugs helped solidify the understanding that ADD had a biological basis even though the exact causal mechanisms were still unknown [0].

4. **Societal and Academic Impact**:
   - The reexamination of ADD led to significant changes in how it was perceived by society and handled in educational settings.
   - As the disorder was better understood, treatment and handling of ADD improved, though complete biological underpinnings remained elusive [0].

5. **Contemporary Understandings and Misunderstandings**:
   - Despite advances, misunderstandings about ADD persisted, particularly regarding its prevalence in adults and the efficacy of the diagnosis.
   - This section explores ongoing debates and the need for further research into ADD to better tailor treatments and interventions [0].

**Case Studies and Personal Accounts**:
   - Narratives such as that of a patient named Joshua, who initially presented with mood disorders, help illustrate the complexities of diagnosing and treating ADD. These personal perspectives underscore the intersection of ADD with other psychological conditions, demonstrating the necessity of nuanced medical and psychological interventions [5].

**Key Takeaways and Lessons Learned**:
   - The transformation in understanding ADD from a moral failing to a medical issue influenced not only treatment strategies but also how individuals with ADD navigate their daily lives and societal expectations.
   - It emphasizes the ongoing need for public and professional education to mitigate stigma and improve the management strategies for ADD.

**Conclusion:**
Chapter 2 provides a historical and cultural context for the understanding of ADD, tracing how perceptions have shifted over time from punitive measures to more scientifically grounded approaches. This narrative not only aids the comprehension of how treatments have evolved but also how societal attitudes have impacted individuals with ADD, making a compelling case for continued education and research in this field.[0] ADHD.txt
[1] ADHD.txt
[2] ADHD.txt
[3] ADHD.txt
[4] ADHD.txt
[5] ADHD.txt


assistant > file_search

assistant > ### Chapter 3: "Sequence Ravelled Out of Sound"

**Title:** "Sequence Ravelled Out of Sound"

**Overview:**  
Chapter 3 delves into the complexities of Attention Deficit Disorder (ADD) in adults, using Emily Dickinson's vivid depiction of a fragmented mind to connect literary expression with the lived experience of ADD. The chapter explores historical perceptions, pivotal studies, and their implications for understanding ADD as a persistent condition into adulthood, rather than a transient childhood disorder.

**Main Sections and Key Points:**

1. **Introduction to Adult ADD**:
   - Emily Dickinson’s description metaphorically captures the distress and fragmented experience often felt by individuals with ADD.
   - The historical misconception that ADD only affects children is discussed, stressing that ADD can continue to complicate adult lives just as significantly [0].

2. **Historical and Clinical Progress**:
   - An important 1978 conference chaired by Leopold Bellak brought forth the acknowledgment that ADD persists into adulthood, a notion that clashed with prevailing perceptions and slowly gained recognition in the following decades [0].
   - Discussions include findings from the conference that marked the beginning of a shift in understanding ADD among medical professionals [0].

3. **Case Studies and Accounts**:
   - Case studies such as the one involving Joshua, who suffered from mood swings that were initially misdiagnosed, highlight the intricate connection between ADD and other emotional disorders, showcasing the depth of misdiagnosis and the complex nature of ADD in adults [3].
   - The narrative emphasizes the long path to correct diagnosis and tailored treatment strategies that significantly improve the quality of life for adults with ADD [3].

4. **The Biological Basis and Further Research**:
   - Dr. Alan Zametkin’s study established a biological basis for ADD by showing differences in brain glucose metabolism between adults with and without ADD [5].
   - This biological insight into ADD paved the way for better-targeted treatments and deeper understanding of the condition beyond mere behavioral analysis [5].

5. **Key Takeaways and Lessons Learned**:
   - The persistence of ADD into adulthood necessitates a reevaluation of how ADD is diagnosed and managed across different stages of life.
   - Awareness and understanding of adult ADD remain crucial for effective management, leading to more scientific approaches and reducing stigma associated with the disorder.

**Conclusion:**
Chapter 3 serves as an enlightening exploration of ADD in adults, dismantling archaic notions and fostering a nuanced understanding of how the disorder impacts the continuum of life from childhood through adulthood. The use of poetic language, historical insights, and empirical evidence crafts a compelling narrative that underscores the enduring nature of ADD and the importance of continued advocacy for affected individuals. This chapter effectively bridges the gap between literary expression and medical discourse, emphasizing the profound impacts of ADD on individuals and the importance of persistent, informed interventions.[0] ADHD.txt
[1] ADHD.txt
[2] ADHD.txt
[3] ADHD.txt
[4] ADHD.txt
[5] ADHD.txt
[6] ADHD.txt


assistant > file_search

assistant > ### Chapter 4: Living and Loving with ADD

**Title:** Living and Loving with ADD

**Overview:**
Chapter 4 delves into the complex dynamics of intimate relationships when one or both partners have Attention Deficit Disorder (ADD). The chapter provides a broad spectrum of insights into how ADD influences relationships, outlining practical strategies for managing these relationships with sensitivity and understanding.

**Main Sections and Key Points:**

1. **Impact on Relationships**:
   - ADD can significantly disrupt intimate relationships, causing daily uncertainties and emotional exhaustion.
   - It often goes unrecognized because symptoms mimic common marital stresses, leading to overlooked diagnoses [0].

2. **Case Studies and Therapeutic Outcomes**:
   - Examples include couples like Sam and Mary Rothman, whose marriage teetered on the brink of divorce due to unidentified ADD. Correct diagnosis and targeted treatment like Ritalin were crucial in helping them manage symptoms and save their marriage [0].
   - Another case involves Peter, whose ADD-related behaviors contributed to severe legal and personal issues, including substance abuse. Treatment for his ADD significantly improved his personal relationships and professional life [2].

3. **Strategies for Managing ADD in Relationships**:
   - Crucial tips include ensuring an accurate diagnosis, maintaining a sense of humor, and creating structured communication times to discuss feelings and manage ADD’s impact on the relationship.
   - These strategies are aimed at recognizing the symptoms of ADD and harnessing strengths to improve relationship dynamics [3].

4. **The Role of Recognition and Understanding**:
   - Recognizing ADD within a relationship context is vital for mitigating misunderstandings and fostering a supportive environment.
   - Awareness and understanding allow couples to navigate the challenges more effectively and use informed interventions to strengthen their bonds [0].

5. **Biological Observations**:
   - ADD symptoms in adults can include impaired self-image, low self-esteem, and secondary symptoms from undiagnosed childhood ADD.
   - These biological insights into ADD help underscore the importance of targeted therapeutic interventions and ongoing management of the condition within personal relationships [0].

**Key Takeaways and Lessons Learned**:
   - Relationships involving individuals with ADD require careful management and a robust understanding of the disorder's effects on both partners.
   - Empathy, education, and structured communication are foundational for navigating the challenges presented by ADD in relationships.
   - Treatment and supportive strategies, while beneficial, do not eliminate the daily challenges of ADD, underscoring the need for persistence and adaptability in approaches to living with and loving someone with ADD.

**Conclusion:**
Chapter 4 provides a compassionate and insightful exploration into the intersection of ADD and intimate relationships. It offers practical tools and narratives that symbolize hope and direction for partners dealing with ADD, emphasizing that while the journey involves continual adjustment, understanding ADD's impacts can lead to healthier, more fulfilling relationships. This chapter is key for anyone involved in such dynamics, offering resources and knowledge to transform challenges into opportunities for growth and understanding.[0] ADHD.txt
[1] ADHD.txt
[2] ADHD.txt
[3] ADHD.txt
[4] ADHD.txt
[5] ADHD.txt


assistant > file_search

assistant > ### Chapter 5: The Big Struggle

**Title:** The Big Struggle

**Overview:**
Chapter 5 details the intense challenges that families often face when managing a child with Attention Deficit Disorder (ADD). It characterizes family conflicts as "The Big Struggle," portraying how these conflicts evolve and impact each family member. The chapter serves as a comprehensive examination of these family dynamics, offering strategies for mitigating the fighting and fostering understanding.

**Main Sections and Key Points:**

1. **Description of the Big Struggle**:
   - The Big Struggle is described as a protracted conflict within families, often resulting in a harsh daily life full of rebukes and misunderstandings.
   - These struggles typically initiate from everyday situations which escalate into larger conflicts, devolving into psychological and emotional strain on all family members .

2. **Case Study: Tommy**
   - Tommy's case illustrates a typical Big Struggle, showing an ongoing conflict between Tommy and his parents stemming from misunderstandings about his behavior and performance in school.
   - This case emphasizes how persistent misunderstandings and conflicts can severely impact a child’s mental health and family dynamics[0].

3. **Biological Insights and Theoretical Concepts**:
   - The chapter scrutinizes ADD’s symptoms such as distractibility and impulsivity, illustrating how they often lead to significant family struggles.
   - It also touches upon the biological underpinnings of ADD, discussing studies by Alan Zametkin and others, which show different brain activity patterns in individuals with ADD compared to those without[1].

4. **Strategies for Managing and Mitigating the Big Struggle**:
   - A range of strategies are recommended, including obtaining an accurate diagnosis, changing the family member’s view on the person with ADD from a problem to someone with unique strengths, and understanding that ADD affects all family members differently[2].
   - The need for effective negotiation within the family and seeking professional help if necessary are also suggested[3].

5. **Empathy and Understanding**:
   - It stresses the importance of empathy and a deep understanding of ADD by all family members to properly support the individual and each other.
   - The chapter emphasizes collaborative problem-solving and consistent, open communication .

**Key Takeaways and Lessons Learned:**
   - ADD is not just an issue of the individual; it affects the entire family dynamics profoundly.
   - Recognizing and addressing ADD can significantly alleviate the Big Struggle, paving the way for a healthier and more understanding family environment.
   - Families are encouraged to shift their approach from blame and punishment to understanding, accommodation, and structured negotiation.

**Conclusion:**
Chapter 5 showcases the complexities and profound impacts of ADD on family life, offering valuable insights into navigating and resolving ongoing family disputes. Through a mix of case studies, theoretical understanding, and practical advice, the chapter provides a pathway for families to turn conflict into collaboration and misunderstanding into mutual respect. The narrative is rich with strategies aimed at diffusing the Big Struggle and fostering an environment of support and resilience within families dealing with ADD.[0] ADHD.txt
[1] ADHD.txt
[2] ADHD.txt
[3] ADHD.txt


assistant > file_search

assistant > ### Chapter 6: Parts of the Elephant

**Title:** Parts of the Elephant

**Overview:**
Chapter 6 addresses the multifaceted nature of Attention Deficit Disorder (ADD) by paralleling it with the story of the blind men describing different parts of an elephant. The chapter discusses the various subtypes of ADD, emphasizing how understanding these subtypes can offer a more comprehensive picture of the disorder.

**Main Sections and Key Points:**

1. **Introduction to Subtypes of ADD**:
   - The chapter starts by likening the complexity of ADD to an ancient story where blind men each touch a different part of an elephant and describe the beast entirely from their limited perspective.
   - This analogy illustrates how viewing only one aspect of ADD can lead to incomplete or misleading perceptions of the disorder[0].

2. **Subtypes of ADD**:
   - The primary subtypes discussed are:
     - ADD without hyperactivity 
     - ADD with hyperactivity
     - ADD with anxiety
     - ADD with depression 
   - These subtypes, though not all officially recognized, help in describing the most common and distinct presentations of ADD, primarily as seen in adults[0].

3. **Biological Observations**:
   - The chapter draws on findings from research to bolster the significant biological underpinnings of ADD.
   - Neurological studies have shown differences in brain activity patterns between those with and without ADD, supporting the subtype classifications[0].

4. **Theoretical Concepts**:
   - Discusses how diverse descriptions of symptoms lead to different therapeutic approaches. 
   - Points out the risk of oversimplification in treating ADD due to its diverse manifestations[0].

5. **Case Studies and Observations**:
   - The text includes hypothetical scenarios and case studies that illustrate how different subtypes of ADD manifest and how they can be effectively managed.
   - These narratives provide concrete examples of the varied nature of ADD and its impacts on individuals' lives[0].

**Key Takeaways and Lessons Learned**:
   - Understanding the diversity within ADD is critical for effective diagnosis and treatment.
   - The chapter emphasizes the importance of taking a holistic view of the disorder rather than focusing too narrowly on specific symptoms or behaviors.
   - It calls for further research and more nuanced clinical approaches tailored to the unique needs of individuals with different subtypes of ADD[0].

**Conclusion:**
Chapter 6 urges clinicians and patients alike to consider ADD as a multifaceted condition with many faces, reminiscent of the different parts of an elephant described by the blind men. By embracing this nuanced perspective, better outcomes can be achieved in both understanding and managing ADD. This holistic approach ensures that treatment strategies are as diverse and complex as the disorder itself, offering more personalized and effective solutions for those affected.[0] ADHD.txt
[1] ADHD.txt
[2] ADHD.txt
[3] ADHD.txt
[4] ADHD.txt
[5] ADHD.txt


assistant > file_search

assistant > ### Chapter 7: How Do I Know if I Have It?

**Title:** How Do I Know if I Have It?

**Overview:**
Chapter 7 delves into the diagnostic process of Attention Deficit Disorder (ADD), examining the subtle yet distinct line between normal behavioral variations and those symptomatic of ADD. It emphasizes the importance of a thorough, detailed personal history and behavioral observations by professionals and close individuals for diagnosis.

**Main Sections and Key Points:**

1. **Steps Toward Diagnosis**:
   - Recognizing the difficulty in clearly distinguishing ADD from normal behavioral variability, emphasizing the gradual, often unclear transition from normal to pathological[0].
   - The diagnosis relies heavily on a detailed personal history, observations by others close to the individual, and the consistency of symptoms over time rather than relying on direct, standalone tests like blood tests or brain imaging[0].

2. **Case Studies Illustrating Diagnosis Challenges and Outcomes**:
   - A case of a woman whose marital and sexual difficulties were finally understood and alleviated with proper diagnosis and treatment for ADD, showcasing dramatic personal improvements after the diagnosis[2].
   - Another significant case involves a man whose multiple unsuccessful psychotherapeutic interventions were reframed and effectively managed once ADD was recognized as the underlying issue[3].

3. **Distinguishing True ADD from Pseudo-ADD**:
   - The chapter discusses the phenomenon of pseudo-ADD, where individuals may relate to the symptoms but do not experience them with the intensity and chronic nature required for a diagnosis of ADD[4].
   - Emphasizes the importance of intensity, duration, and functional impairment in distinguishing true ADD from normal variations or less severe symptom presentations.

4. **Importance of Professional Evaluation**:
   - Stresses the integral role of professional evaluation in diagnosing ADD to prevent misdiagnosis and ensure that other possible conditions are considered or ruled out[5].
   - Highlights the risks of self-diagnosis and the need for clinical confirmation through structured interviews and possibly supplementary tests if indicated[6].

**Key Takeaways and Lessons Learned**:
   - Diagnosis of ADD is a nuanced process that hinges on detailed personal histories and requires careful differentiation from normal behavioral traits.
   - Professional involvement is crucial not only for accurate diagnosis but also for the management of ADD, nodding to the complexity and potential overlap with other psychological and physiological conditions.
   - The chapter underscores the potential for significant life improvements with correctly identified and managed ADD, thereby improving individual confidence in personal and professional spheres[0].

**Conclusion:**
Chapter 7 effectively demystifies the diagnosis process for ADD, equipping readers with an understanding of both the complexity involved and the critical nature of professional guidance. It serves as a crucial guide for individuals suspecting they or loved ones might have ADD, encouraging meticulous evaluation and caution against self-diagnosis, thereby fostering more accurate recognition and empowerment for effective management of ADD.[0] ADHD.txt
[1] ADHD.txt
[2] ADHD.txt
[3] ADHD.txt
[4] ADHD.txt
[5] ADHD.txt
[6] ADHD.txt
[7] ADHD.txt


assistant > file_search

assistant > ### Chapter 8: What Can You Do About It?

**Title:** What Can You Do About It?

**Overview:**
Chapter 8 provides an in-depth exploration of the various management strategies for Attention Deficit Disorder (ADD), addressing multiple methods from medication and therapy to personal coping strategies and organizational techniques. This chapter serves as a comprehensive guide for individuals with ADD, their families, and professionals, offering practical advice for coping with and managing the symptoms effectively.

**Main Sections and Key Points:**

1. **Overview of Management Strategies**:
   - Introductory remarks emphasize the complexity of ADD and the need for a multifaceted management approach that includes medication, cognitive strategies, lifestyle changes, and possible therapy[0].

2. **Medications and Clinical Treatments**:
   - The chapter discusses various pharmacological treatments used to manage ADD, noting the effectiveness and side effects of stimulants and non-stimulants.
   - Clinical treatments including behavioral therapy and counseling are crucial, especially when combined with medication[0].

3. **Personal Coping Mechanisms**:
   - Practical advice on organizing daily activities to minimize the impact of ADD on productivity and personal relations, such as making lists, using bulletin boards, and writing notes[0].
   - Strategies like breaking down tasks into manageable parts and creating a structured daily routine are recommended to help maintain focus and reduce overwhelm[0].

4. **Cognitive and Behavioral Strategies**:
   - Focused attention on cognitive-behavioral techniques such as mindfulness, time management, and prioritization which help mitigate distractibility and impulsivity[0].
   - Emphasizes the importance of self-acceptance and understanding the individual nature of one’s ADD symptoms[0].

5. **Family and Relationship Support**: 
   - The role of family and close relationships in managing ADD is addressed, advocating for open communication and mutual understanding to create supportive environments[0].
   - Guidelines for families and partners on living with someone with ADD, stressing the importance of patience, structured interaction, and joint management plans[0].

**Key Takeaways and Lessons Learned**:
   - Managing ADD requires a holistic approach that involves medical, personal, cognitive, and social strategies.
   - The guidance provided is aimed to empower individuals with ADD to lead organized, fulfilled lives, acknowledging that while challenges persist, effective management is indeed possible.
   - Education and understanding are emphasized as critical aspects of successful management of ADD, both for individuals and their support systems[0].

**Conclusion:**
Chapter 8 equips readers with a robust set of tools and knowledge for managing ADD, emphasizing the necessity of personalized strategies tailored to individual needs. From detailed medicinal overviews to tips on daily living, the chapter provides invaluable resources, ensuring individuals with ADD and their loved ones have access to the necessary support for thriving despite the challenges ADD may pose.[0] ADHD.txt
[1] ADHD.txt
[2] ADHD.txt
[3] ADHD.txt
[4] ADHD.txt
[5] ADHD.txt
[6] ADHD.txt
[7] ADHD.txt
[8] ADHD.txt


assistant > file_search

assistant > ### Chapter 9: A Local Habitation and a Name

**Title:** A Local Habitation and a Name

**Overview:**
Chapter 9 delves deeply into the biological underpinnings of Attention Deficit Disorder (ADD), bridging past and contemporary research with a focus on how ADD resides within the complex network of the brain's structures and chemical interactions. This chapter shifts from symptom observation to exploring the neurobiological bases that contribute to the behavioral manifestations of ADD.

**Main Sections and Key Points:**

1. **The Biological Basis of ADD:**
   - Details how ADD is fundamentally rooted in the biology of the brain and the central nervous system, emphasizing that symptoms stem from neurobiological malfunctioning[0].
   - Discusses the ongoing challenge in pinpointing a single cause due to the complexity of the brain's attentional system which involves nearly all brain structures[0].

2. **Historical Perspectives on ADD:**
   - Traces the evolution of understanding ADD, marking the shift from moralistic views to scientific inquiry that recognizes the biological factors influencing ADD[0].
   - Highlights significant milestones, like the use of stimulants which paradoxically aid focus in those with ADD, contributing to a biological understanding of the disorder[0].

3. **Research and Discoveries:**
   - Explains how the field has grown through both successes and misleads in research, and how ADD has been historically challenging to define due to its diverse manifestations and similarities with other disorders[0].

4. **Implications for Treatment and Perception:**
   - Emphasizes that recognizing the biological basis of ADD shifts treatment approaches from punitive to therapeutic, focusing more on medical interventions and understanding rather than moral judgment[0].
   - Advocates for ongoing research to further clarify the biological pathways involved in ADD, improving diagnosis and treatment strategies[0].

**Key Takeaways and Lessons Learned:**
   - The chapter underscores the importance of understanding ADD as a neurobiological condition, which helps in reducing stigma and improving targeted treatment approaches.
   - It also calls for a compassionate understanding that individuals with ADD are not deficient in moral fiber or will, but are experiencing a medical condition that affects their cognitive and behavioral responses.
   - Reflects on the progress made in understanding ADD while also acknowledging the vast amount still to be learned in terms of its precise biological mechanisms[0].

**Conclusion:**
Chapter 9 provides a detailed exploration of the biological landscape of ADD, integrating historical and current research to present a comprehensive view of the disorder as a complex neurological condition. By illuminating the biological foundations of ADD, this chapter contributes crucial insights that advance both clinical practice and societal perceptions, fostering a more informed and empathetic approach to managing ADD.[0] ADHD.txt
[1] ADHD.txt
[2] ADHD.txt
[3] ADHD.txt
[4] ADHD.txt
[5] ADHD.txt
[6] ADHD.txt
[7] ADHD.txt


assistant > file_search

assistant > ### Chapter 10: The Biology of ADD

**Title:** The Biology of ADD

**Overview:**
Chapter 10 delves deeply into the biological underpinnings of Attention Deficit Disorder (ADD), emphasizing research findings on brain function and neurochemical dynamics. The chapter discusses significant studies and advancements in understanding ADD’s biological basis, correlating these findings with the symptoms and treatment approaches for the disorder.

**Main Sections and Key Points:**

1. **Biological Basis of ADD**:
   - Explores how ADD is associated with differential brain activity, particularly in the utilization of glucose during cognitive tasks, highlighting a decrease in glucose metabolism in certain brain regions among people with ADD compared to controls[0].

2. **Neurotransmitter Activity and Hypotheses**:
   - Discusses the catecholamine hypothesis of ADD, which suggests that alterations in dopamine and norepinephrine levels may underlie many of the disorder's symptoms. This theory is supported by studies showing disrupted neurotransmitter activity in individuals with ADD[1].
   - Examination of the role of other neurotransmitters such as serotonin, and how their interaction might also play a role in the manifestation of ADD symptoms[0].

3. **Case Study: Impact of Medication on Treatment**:
   - Outlines a case study involving the use of stimulant medication, demonstrating how these drugs can significantly improve brain function related to attention and impulse control, corroborating the biological basis of ADD[3].
   - Discussion on the mediated effects of neurotransmitter regulation through medication, leading to enhanced cognitive function and behavioral control[4].

4. **Future Directions in Research**:
   - Calls for more nuanced research into specific neurotransmitters and brain regions to refine understanding and treatment of ADD.
   - Acknowledges the complexity of ADD’s neurobiological profile and advocates for multidisciplinary approaches to further dissect the precise mechanisms at play[1].

**Key Takeaways and Lessons Learned:**
   - Establishes a clear link between the biological aspects of ADD and its clinical symptoms, underscoring the importance of a biological perspective in understanding and treating the disorder.
   - Highlights the potential for targeted pharmacological interventions that can modulate neurotransmitter activity, thus mitigating the core symptoms of the disorder.
   - Emphasizes the need for continued research to explore the detailed interactions of brain chemistry and structure to better tailor treatments to individuals with ADD[0].

**Conclusion:**
Chapter 10 provides a comprehensive overview of the biological foundations of ADD, using landmark studies and current research to paint a detailed picture of the neurological and biochemical dynamics underpinning the disorder. This biological perspective is crucial for developing effective treatment strategies and for continuing to reduce the stigma associated with ADD by framing it within a medical model.[0] ADHD.txt
[1] ADHD.txt
[2] ADHD.txt
[3] ADHD.txt
[4] ADHD.txt
[5] ADHD.txt
[6] ADHD.txt



In [ ]:
def execute_stream(prompt):
    
    message = client.beta.threads.messages.create(
        thread_id=thread.id, 
        role="user", 
        content=prompt, 
    )
    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please address the user as 'my dear reader and friend'. The user has a premium account.",
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()
        

In [56]:
prompt = """Can you tell me in detail about the cases discussed in Chapter 4?"
"""
        
display_handle = display(Markdown(""), display_id=True)
message_content = ""
execute_stream(prompt)

# result = execute_run(client, assistant, thread)



assistant > file_search

assistant > 
Certainly, my dear reader and friend, chapter 4 of the text you are exploring details intriguing cases that emphasize the significant challenges and triumphs related to Attention Deficit Disorder (ADD) in both individual and familial contexts. Here's a detailed look at the cases:

1. **Case of Danny and David:**
   - **Background:** Danny and David, initially misdiagnosed and placed in highly restrictive educational settings due to presumed low IQ and emotional disturbances, faced severe educational and social challenges.
   - **Intervention:** The twins' persistent issues were ultimately recognized as severe ADD. They were removed from the special school setting and introduced to a less restrictive environment where their treatment included the administration of Ritalin.
   - **Outcomes:** Remarkable improvement was observed both in school behavior and academic performance. Post-diagnosis and treatment initiation, they were able to join mainstream educational settings. They excelled in school and developed notable talents in music and martial arts, highlighting the profound impact of appropriate diagnosis and treatment[0].

2. **Case of Peter:**
   - **Background:** Peter faced challenges with substance abuse and legal issues, which were later connected to his undiagnosed ADD. 
   - **Intervention:** After being released from jail, he was diagnosed with ADD and began treatment, which included medication.
   - **Outcomes:** The treatment radically altered his trajectory, reducing his urge for substance abuse and enabling him to excel academically and professionally. His relationships and quality of life improved significantly as a direct result of his treatment for ADD[1].

These detailed cases illuminate not only the profound struggles faced by individuals with ADD but also highlight the critical role of accurate diagnosis and tailored treatment plans. They serve as poignant illustrations of how proper medical attention can drastically improve the lives of those affected by ADD, resulting in significant improvements in social, academic, and personal areas.[0] ADHD.txt
[1] ADHD.txt



In [58]:
prompt = """Can you create a glossary with definitions of all the technical terms I should know when reading this text?
Please extract and synthesize all the definitions and explanations of terms from across all chapters of the book.
"""
        
display_handle = display(Markdown(""), display_id=True)
message_content = ""
execute_stream(prompt)



assistant > file_search

assistant > 
Certainly, my dear reader and friend. Let us synthesize a glossary of technical terms explored across the various chapters of the book. This compilation will enhance your comprehension as you read through the text:

1. **ADD (Attention Deficit Disorder)**: A neurobehavioral condition characterized by chronic issues such as impulsiveness, short attention span, and, in some cases, hyperactivity that interfere significantly with daily functioning and development .

2. **Hyperactivity**: A condition marked by excessive movement and actions that may be non-purposeful and disruptive. In the context of ADD, it's one of the possible symptoms where the affected person may feel restless and have difficulty sitting still .

3. **Distractibility**: Indicates a higher than usual tendency to pay attention to extraneous stimuli or to shift attention frequently from one activity to another .

4. **Impulsivity**: A behavioral characteristic that entails making quick actions that occur in the moment without forethought and that may have high potential for harm .

5. **Catecholamine Hypothesis of Hyperactivity**: Proposes that problems like ADD may be due to an underproduction of neurotransmitters like norepinephrine and dopamine in the brain, affecting attention and behavior control .

6. **Learning Disorders**: Refers to problems in an individual’s neurocognitive functioning which significantly affect academic performance, such as in reading (dyslexia), writing, and mathematics. ADD is considered within this spectrum when it impairs learning capacity .

7. **Neurotransmitters**: Chemicals in the brain involved in transmitting signals between nerve cells. Notable ones involved in ADD are dopamine and norepinephrine, which affect focus, memory, and emotional responses .

8. **Stimulant Medication**: Drugs, primarily used to treat ADD, that stimulate brain activity, helping to increase attention and reduce impulsivity and hyperactivity. Examples include Ritalin and Adderall .

9. **Behavioral Therapy**: A type of psychotherapy that aims to modify harmful behaviors through techniques such as reinforcement (encouraging desired behaviors through rewards) and behavior modification strategies .

This glossary should provide a basis for understanding the central concepts when reading about ADD in the text provided. Each term has been pulled from general discussions across various sections of the book, helping bridge the scientific explanations with the narrative descriptions you will encounter.



In [59]:
prompt = """What types of medications are used to treat ADHD?"
"""
display_handle = display(Markdown(""), display_id=True)
message_content = ""
execute_stream(prompt)



assistant > file_search

assistant > 
My dear reader and friend, the treatment for Attention Deficit Disorder (ADD) involves a variety of medications, each tailored to manage specific symptoms and improve focus, impulse control, and overall functioning. Let me elucidate the types of medications that are commonly used to treat ADD based on the comprehensive details found in the text:

1. **Stimulants**: 
   - These are the most widely used medications for managing ADD. They work by activating neurotransmitters such as dopamine and norepinephrine which help improve focus, attention, and control impulsive behaviors.
   - Common stimulants include:
     - **Ritalin (methylphenidate)**: Available in short-acting and long-acting forms such as Ritalin LA, and other variants like Methylin and Metadate[0].
     - **Dexedrine**: Known for treating ADD with options for different forms for extended effects[0].
     - **Adderall and Adderall XR**: Effective for prolonged treatment periods[0].
     - **Vyvanse**: Considered a purer form of amphetamine[0].

2. **Non-stimulant Medications**:
   - **Strattera**: Works on neurotransmitters to help increase attention and control hyperactivity[4].
   - **Antidepressants**: Used when ADD is accompanied by anxiety or depression. These include:
     - **Wellbutrin**
     - **Norpramin**
     - **Tricyclic antidepressants** like Tofranil, which can also address symptoms of ADD[4].

3. **Mood Stabilizers and Other Adjunctive Medications**:
   - For cases where mood dysregulation is significant, mood stabilizers such as valproic acid (Depakote) and lithium might be used[6].
   - **Beta-blockers** like nadolol and propranolol can be prescribed to manage the impulsivity and explosive temper often associated with ADD[6].
   - Serotonin-active drugs like fluoxetine might be added particularly for women who experience PMS symptoms exacerbating their ADD symptoms[8].

4. **Additional Considerations**:
   - It is essential to monitor the use of these medications under medical supervision to adjust dosages correctly and manage any potential side effects, which can range from appetite suppression and sleep disturbances to more psychological symptoms like mood changes[0].

These treatments highlight the complexity and individualized nature of managing ADD, necessitating a comprehensive approach that not only addresses the neurochemical imbalances but also considers the holistic impact on an individual’s life[0]. As you navigate through your readings and possible treatments, these insights should provide a clearer understanding of how various medications function and the importance of targeted treatment strategies for managing ADD effectively.[0] ADHD.txt
[1] ADHD.txt
[2] ADHD.txt
[3] ADHD.txt
[4] ADHD.txt
[5] ADHD.txt
[6] ADHD.txt
[7] ADHD.txt
[8] ADHD.txt
[9] ADHD.txt
[10] ADHD.txt



In [60]:
prompt = """What are the different types of antidepressants, how do they work and how do they help patients with ADHD?"
"""
display_handle = display(Markdown(""), display_id=True)
message_content = ""
execute_stream(prompt)



assistant > file_search

assistant > 
My dear reader and friend, let's explore the different types of antidepressants, their mechanisms, and how they can assist patients with Attention Deficit Disorder (ADD), specifically those who may also grapple with symptoms of depression.

### Types of Antidepressants:

1. **Strattera (Atomoxetine)**:
   - Mechanism: Unlike typical antidepressants, Strattera is not only used as an antidepressant but also one of the prominent non-stimulant medications for ADD. It primarily influences the neurotransmitter norepinephrine and enhances attention and decreases hyperactivity and impulsivity.
   - Usage in ADD: It is commonly preferred for patients who might not respond well to or tolerate stimulant medications[0].

2. **Wellbutrin (Bupropion)**:
   - Mechanism: Acts as a norepinephrine-dopamine reuptake inhibitor (NDRI). It increases the levels of these neurotransmitters in the brain to help improve concentration, energy, and mood.
   - Usage in ADD: Often used to treat ADD due to its stimulating effects on focus and control of impulsivity, and it can also address co-existing depressive symptoms[0].

3. **Norpramin (Desipramine)**:
   - Mechanism: Falls under the category of tricyclic antidepressants, which work by inhibiting the reuptake of certain neurotransmitters like norepinephrine and serotonin, thus increasing their levels in the brain.
   - Usage in ADD: Helps in the management of attentional symptoms and can also be effective for patients with concurrent anxiety or depression[0].

4. **SSRIs (Selective Serotonin Reuptake Inhibitors) such as Prozac (Fluoxetine) and Zoloft (Sertraline)**:
   - Mechanism: These medications increase the level of serotonin in the brain, a neurotransmitter that can affect mood, emotion, sleep, and appetite.
   - Usage in ADD: While primarily used for depression and anxiety disorders, they are sometimes used in ADD patients to manage overlapping emotional or depressive symptoms that can commonly accompany ADD[0].

### How Antidepressants Help Patients with ADD:

Antidepressants can be beneficial in the treatment of ADD for several reasons:

- **Mood Stabilization**: Many patients with ADD experience mood swings or emotional dysregulation. Antidepressants can help stabilize mood, thereby reducing the intensity of emotional reactions and the accompanying distraction they can cause[0].
  
- **Improvement in Focus and Reduction in Impulsivity**: Especially true for antidepressants like Wellbutrin and Norpramin, which have been shown to enhance focus and decrease impulsivity, critical challenges in ADD[0].
  
- **Management of Co-existing Disorders**: ADD often coexists with other disorders such as depression and anxiety. The broad-spectrum action of antidepressants not only helps manage ADD but also addresses these concurrent conditions, providing a more comprehensive treatment approach[0].

In conclusion, while stimulants are more commonly associated with the primary treatment of ADD, antidepressants are invaluable, especially when treating complex cases where mood disorders coexist or when the stimulant treatments are inadequate or unsuitable due to side effects or other concerns. The choice of antidepressant, as always, should be guided by detailed individual assessments by healthcare professionals[0].[0] ADHD.txt
[1] ADHD.txt
[2] ADHD.txt
[3] ADHD.txt
[4] ADHD.txt
[5] ADHD.txt
[6] ADHD.txt
[7] ADHD.txt



In [62]:
prompt = """Please tell me in detail about the cases of George, Grace, Peter, Elizabeth, and Harry and which chapters they are discussed in?"
"""

display_handle = display(Markdown(""), display_id=True)
message_content = ""
execute_stream(prompt)



assistant > file_search

assistant > 
My dear reader and friend, I am here to present detailed descriptions of the cases involving George, Grace, Peter, Elizabeth, and Harry as they appear across various chapters in your text:

1. **George**:
   - **Chapter Discussed**: Not specified.
   - **Details**: George struggled with his familial relationships, particularly with seeking approval from his sister, which he never received. His journey involves understanding his ADD and trying to explain this to his family to foster a better support system[0].

2. **Grace**:
   - **Chapter Discussed**: Not specified.
   - **Details**: Grace relishes the chaos that ADD brings into her life. Working in the movie industry, she considers her ability to multitask and her impulsive decisions as assets that she wouldn’t change even though they are symptoms of ADD[1].

3. **Peter**:
   - **Chapter Discussed**: Chapter 4 and possibly Chapter 9 
   - **Details**: Peter’s case is particularly striking as he ended up in jail due to issues arising from his undiagnosed ADD. Post-diagnosis, he could turn his life around significantly by reducing his impulse for drug abuse and engaging more constructively in society[2].

4. **Elizabeth**:
   - **Chapter Discussed**: Not specified.
   - **Details**: Elizabeth struggled with dyslexia and ADD, which went unrecognized for years. She always considered herself disorganized and inept until she learned about ADD. This understanding finally brought clarity to many aspects of her life that she struggled with[3].

5. **Harry**:
   - **Chapter Discussed**: Not specified.
   - **Details**: Harry, now a successful businessman, revisited the painful memories of his schooling years from a file kept by his school’s rabbi. Understanding his ADD later in life provided him with great relief and answers to why he struggled academically and socially during his younger years[4].

Each case provides a profound look at the impacts of ADD on different aspects of life and the transformative power of diagnosis and understanding. These narratives serve to enrich the comprehension of ADD’s broad and diverse effects on individuals’ lives, highlighting the need for awareness and supportive environments.[0] ADHD.txt
[1] ADHD.txt
[2] ADHD.txt
[3] ADHD.txt
[4] ADHD.txt



In [64]:
messages = client.beta.threads.messages.list(thread_id=thread.id)


In [68]:
len(messages.data)

20

In [76]:
messages.data[0].content[0].text.annotations

[FileCitationAnnotation(end_index=525, file_citation=FileCitation(file_id='file-c1ZqSWHuC2WwyYqgsGG7yEtI', quote=None), start_index=510, text='【69:0†ADHD.txt】', type='file_citation'),
 FileCitationAnnotation(end_index=843, file_citation=FileCitation(file_id='file-c1ZqSWHuC2WwyYqgsGG7yEtI', quote=None), start_index=828, text='【69:4†ADHD.txt】', type='file_citation'),
 FileCitationAnnotation(end_index=1206, file_citation=FileCitation(file_id='file-c1ZqSWHuC2WwyYqgsGG7yEtI', quote=None), start_index=1191, text='【69:1†ADHD.txt】', type='file_citation'),
 FileCitationAnnotation(end_index=1557, file_citation=FileCitation(file_id='file-c1ZqSWHuC2WwyYqgsGG7yEtI', quote=None), start_index=1542, text='【69:2†ADHD.txt】', type='file_citation'),
 FileCitationAnnotation(end_index=1930, file_citation=FileCitation(file_id='file-c1ZqSWHuC2WwyYqgsGG7yEtI', quote=None), start_index=1915, text='【69:6†ADHD.txt】', type='file_citation')]

In [96]:
# I think start_index and end_index might refer to where the annotation is in the response.abs
# not sure how to decode the citation and get relevant chunks from the text
# Open the file and read it into a string

with open("ADHD.txt", 'r', encoding='utf-8') as file:
    file_contents = file.read()
    
message_content = messages.data[0].content[0].text
annotations = message_content.annotations
citations = []
 
# Iterate over the annotations and add footnotes
for index, annotation in enumerate(annotations):

    print(index, annotation, annotation.text)
    # Replace the text with a footnote
    message_content.value = message_content.value.replace(annotation.text, f' [{index}]')
 
    # Gather citations based on annotation attributes
    if (file_citation := getattr(annotation, 'file_citation', None)):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f'[{index}] {file_citation.quote} from {cited_file.filename}')
        print(file_contents[annotation.start_index:annotation.end_index])
    elif (file_path := getattr(annotation, 'file_path', None)):
        cited_file = client.files.retrieve(file_path.file_id)
        citations.append(f'[{index}] Click <here> to download {cited_file.filename}')
        # Note: File download functionality not implemented above for brevity
 
# Add footnotes to the end of the message before displaying to user
message_content.value += '\n' + '\n'.join(citations)

0 FileCitationAnnotation(end_index=525, file_citation=FileCitation(file_id='file-c1ZqSWHuC2WwyYqgsGG7yEtI', quote=None), start_index=510, text='【69:0†ADHD.txt】', type='file_citation') 【69:0†ADHD.txt】
on

Delivered f
1 FileCitationAnnotation(end_index=843, file_citation=FileCitation(file_id='file-c1ZqSWHuC2WwyYqgsGG7yEtI', quote=None), start_index=828, text='【69:4†ADHD.txt】', type='file_citation') 【69:4†ADHD.txt】
t,

Lengthen Yo
2 FileCitationAnnotation(end_index=1206, file_citation=FileCitation(file_id='file-c1ZqSWHuC2WwyYqgsGG7yEtI', quote=None), start_index=1191, text='【69:1†ADHD.txt】', type='file_citation') 【69:1†ADHD.txt】
nterruption

(w
3 FileCitationAnnotation(end_index=1557, file_citation=FileCitation(file_id='file-c1ZqSWHuC2WwyYqgsGG7yEtI', quote=None), start_index=1542, text='【69:2†ADHD.txt】', type='file_citation') 【69:2†ADHD.txt】

What Are You W
4 FileCitationAnnotation(end_index=1930, file_citation=FileCitation(file_id='file-c1ZqSWHuC2WwyYqgsGG7yEtI', quote=None), start_inde

In [94]:
# Open the file and read it into a string
file_contents[828:843]

't,\n\nLengthen Yo'